In [1]:
let char_list_to_string cl = String.concat "" (List.map (String.make 1) cl)

val char_list_to_string : char list -> string = <fun>


In [9]:
char_list_to_string [' '; 'a'; ' ';]

- : string = " a "


In [6]:
let string_to_char_list niz = List.init (String.length niz) (String.get niz)

val string_to_char_list : string -> char list = <fun>


In [8]:
string_to_char_list " ab c"

- : char list = [' '; 'a'; 'b'; ' '; 'c']


In [10]:
String.trim "   abc    "

- : string = "abc"


In [11]:
let trak1 = "   abcDe  fg    h    "
(* Imam torej kao glavo na D. in mam en seznam, k je c b a prazen, in morm sam to nakonc porezat, pa obrnt.*)
(* Lahko levga najprej obrnem, potem najdem find_index od prvega elementa, k ni prazen in pol nardim sub
    Desn pa morm najt zadnga, da so od tam naprej sam še presledki ... nočm ga za brez veze obračat ...
    Ker more bit kao učinkovito to vse skupi .... rabla bi sam en left/right trim ...
    Okej pač enkrat se bom mogla čez sprehodt. Grem se probat sam enkrat.
    Za ta desnga. Grem enkrat čez, da dobim indeks tazadnga k je blank z eno funkcijo in pol substring.
    Grem torej dvakrat čez. Kar je bolš, kokr da dvakrat obrnem in enkrat iščem.
    Za ta levga pa pač obrnem in grem do tam, kjer prvič je znak.
    Kaj pa če mam desn seznam in grem čez, da najdem zadnjega praznega
in potem še enkrat čez sam do unga indeksa z nekim mapi in hkrati nardim string.
Pa a se pozna, da nardim tri stringe in jih concateniram skupi, kokr pa da dodajam na enga?
Sej ne vem, kako stringi delujejo - a je sploh razlika?*)

val trak1 : string = "   abcDe  fg    h    "


In [12]:
String.split_on_char ' ' trak1

- : string list =
[""; ""; ""; "abcDe"; ""; "fg"; ""; ""; ""; "h"; ""; ""; ""; ""]


In [18]:
let pomozna_desni1 desni_char_list =
  let rec aux i_konca i carry sez =
    match carry, sez with
    | _, [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | true, x :: xs when x = ' ' -> aux i_konca (i + 1) true xs (* vrsta presledkov *)
    | false, x :: xs when x = ' ' -> aux i (i + 1) true xs (*vidu prvi presledek po drugih znakih*)
    | carry, x :: xs -> aux (i + 1) (i + 1) false xs (*kerkol drug znak pa izklop zastavco, pa ta znak mora bit v nizu, tkoda i + 1
    ker bo šlo do ne-vključno. Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.
    Pa ne rabm zastavce ?*)
  in
  aux 0 0 false desni_char_list

val pomozna_desni1 : char list -> int = <fun>


In [19]:
pomozna_desni1 (string_to_char_list " ab c   ")

- : int = 5


In [26]:
pomozna_desni1 (string_to_char_list " ")

- : int = 0


In [23]:
let pomozna_desni2 desni_char_list =
  let rec aux i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux i_konca (i + 1) xs (* vrsta presledkov *)
    | x :: xs -> aux i (i + 1) xs (*kerkol drug znak pa izklop zastavco, pa ta znak mora bit v nizu, tkoda i + 1
    ker bo šlo do ne-vključno. Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.
    Pa ne rabm zastavce ?*)
  in
  aux 0 0 desni_char_list

val pomozna_desni2 : char list -> int = <fun>


In [32]:
pomozna_desni2 (string_to_char_list "     ")

- : int = 0


In [ ]:
let pomozna_desni desni_char_list =
  let rec aux_iskanje_konca i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux_iskanje_konca i_konca (i + 1) xs
    | x :: xs -> aux_iskanje_konca i (i + 1) xs (*Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.*)
  in
  let indeks_konca_niza = aux_iskanje_konca 0 0 desni_char_list in
  let zberi_začetek_v_niz i_konca cl =
    let rec aux_v_niz acc i l =
      match i, l with
      | _, [] -> acc
      | i, x :: xs when i <= i_konca -> aux_v_niz (acc ^ Char.escaped x) (i + 1) xs
      | i, x :: xs -> acc (* Nas ne zanima več*)
      (* to men zgleda zelo ne dobr časovno, ker gre kvadratično čez string
      Sam kako pa list.concat to nardi? ...... ???*)
    in
    aux_v_niz "" 0 cl
  in
  zberi_začetek_v_niz indeks_konca_niza desni_char_list
  (*hotla sm neki z List.mapi in pol vrže ven vse ostale.
  Une spremeni v string, tisto pa vrže ven, kar je odveč. A probam?*)
in
let desni_niz = pomozna_desni desni_char_list in

In [36]:
let filter_mapi f =
  let rec aux acc i =
    function
    | [] -> List.rev acc
    | x :: xs -> (
      if Option.is_none (f i x) then aux acc (i + 1) xs
      else aux (Option.get (f i x) :: acc) (i + 1) xs
    )
  in
  aux [] 0

val filter_mapi : (int -> 'a -> 'b option) -> 'a list -> 'b list = <fun>


In [39]:
(* Probam še z vgrajenimi bolj zale konc*)
let pomozna_desni desni_char_list =
  let rec aux_iskanje_konca i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux_iskanje_konca i_konca (i + 1) xs
    | x :: xs -> aux_iskanje_konca i (i + 1) xs (*Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.*)
  in
  let indeks_konca_niza = aux_iskanje_konca 0 0 desni_char_list in
  let f_desni_map_z_indeksom_konca i c =
    if i <= indeks_konca_niza then Some (Char.escaped c)
    else None
  in
  filter_mapi f_desni_map_z_indeksom_konca desni_char_list
  |> String.concat ""
  (* ubistvu bi rabla filter_mapi, ki pa ne obstaja, ampak smo ga enkrat implementiral ...*)
  (*hotla sm neki z List.mapi in pol vrže ven vse ostale.
  Une spremeni v string, tisto pa vrže ven, kar je odveč. A probam?*)

val pomozna_desni : char list -> string = <fun>


In [41]:
let cl1 = [' '; ' '; 'a'; 'b'; ' '; 'c'; ' ']

val cl1 : char list = [' '; ' '; 'a'; 'b'; ' '; 'c'; ' ']


In [42]:
pomozna_desni cl1

- : string = "  ab c"
